# this file use to create labels mask from spm12 by concat all tissue type in 1 labal mask

In [ ]:
import os
from pathlib import Path
import pandas as pd
import auxiliary as aux_func
import numpy as np
import cv2
import matplotlib.pyplot as plt
import nibabel as nib

PIXEL_THRESHOLD = 100

ax3_title = {
    "v4": "brain (windowing > 300 then filled contours)",
    "v5": "brain (windowing > 0 && < 90)"
}
MAP_COLORS = np.array(
    [
        [0,   0,   0],  # black
        # [215, 175, 168], # brownish pink
        [56, 180, 139],  # green
        [50,  76, 172],  # dark blue
        # [ 66, 255, 255], # cyan
    ], dtype="uint8")


def plot_compair_each_slice(gt_slice, brain_and_csf_slice, brain_title, save_path):
    mask = np.where(brain_and_csf_slice[..., np.newaxis] == 2,
                    gt_slice[..., np.newaxis] + MAP_COLORS[2], gt_slice[..., np.newaxis])
    mask = np.where(brain_and_csf_slice[..., np.newaxis] ==
                    1, gt_slice[..., np.newaxis] + MAP_COLORS[1], mask)
    mask[mask > 255] = 255

    fig = plt.figure(figsize=(10, 5))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.axis("off")
    ax1.set_title("gt")
    ax1.imshow(gt_slice, cmap="gray")

    ax2 = fig.add_subplot(1, 2, 2)
    ax2.set_title(brain_title)
    ax2.axis("off")
    ax2.imshow(mask)

    fig.savefig(save_path)
    plt.close(fig)


# ratio with spm12
def cal_csf_to_brain_ratio_v1(gt, gray, white, csf, bone, is_plot, save_path=None, *args, **kwargs):
    brain = (gray | white | csf) - bone
    brain = np.where(brain < 0, 0, brain)
    new_mask = np.zeros_like(gt, dtype="uint8")
    csf_count = np.sum(csf)
    brain_count = np.sum(brain)
    csf_brain_ratio = csf_count / brain_count
    new_mask = np.where(bone > 0, 2, new_mask)  # order matter
    new_mask = np.where(csf > 0, 1, new_mask)
    if is_plot:
        new_file_name = ("%s.nii" % (file_name))
        new_mask_file = save_path / new_file_name
        nib.save(nib.Nifti1Image(new_mask, affine=gt_nifti.affine,
                 header=gt_nifti.header), new_mask_file)

    return csf_brain_ratio, csf_count, brain_count, gray.shape[2], gray.shape[2]


# ratio with spm12 and plot
def cal_csf_to_brain_ratio_v2(gt, gray, white, csf, bone, *args, is_plot, save_path=None, **kwargs):
    if is_plot and not save_path:
        raise AttributeError("if is_plot is true then save_path must given")

    brain_count = 0
    csf_count = 0
    gray_white_csf_spm = gray | white | csf
    for i in range(gt.shape[2]):
        brain_slice = gray_white_csf_spm[..., i]
        brain_from_spm = np.sum(brain_slice)
        brain_count += brain_from_spm
        ax3_title = "brain (g+w+c)"

        if is_plot:
            new_save_path = save_path / ("%d.png" % i)
            mask = np.where(brain_slice == 1, 2, 0)
            mask = np.where(csf[..., i] == 1, 1, mask)
            plot_compair_each_slice(
                gt[..., i], brain_slice, ax3_title, str(new_save_path))

    csf_brain_ratio = csf_count / brain_count
    return csf_brain_ratio, csf_count, brain_count, gt.shape[2], gt.shape[2]


def cal_csf_to_brain_ratio_v3(gt, gray, white, csf, bone, *args, is_plot=False, save_path=None):
    if is_plot and not save_path:
        raise AttributeError("if is_plot is true then save_path must given")

    brain_count = 0
    csf_count = 0
    ok_slice = 0
    for i in range(gt.shape[2]):
        print(i, end=" ", flush=True)
        gray_white_i = gray[..., i] | white[..., i]
        contoured_bone = processing_bone(bone[..., i])
        brain_from_contoured_bone = np.sum(contoured_bone)

        if np.sum(gray_white_i) > PIXEL_THRESHOLD:  # pixels threshold
            ok_slice += 1
            brain_count += brain_from_contoured_bone
            csf_count += np.sum(csf[..., i])
            if is_plot:
                new_save_path = save_path / ("%d.png" % i)
                ax3_title = "brain (contours bone)"
                mask = np.where(contoured_bone == 1, 2, 0)
                mask = np.where(csf[..., i] == 1, 1, mask)
                plot_compair_each_slice(
                    gt[..., i], mask, ax3_title, str(new_save_path))

    if brain_count != 0:
        csf_brain_ratio = csf_count / brain_count
    else:
        csf_brain_ratio = 0
    return csf_brain_ratio, csf_count, brain_count, ok_slice, gt.shape[2]


def cal_csf_to_brain_ratio_v4(gt, gray, white, csf, bone, *args, is_plot=False, save_path=None, file_name=None, **kwargs):
    # expected bone that has been windowing from gt >= 300
    if is_plot and not save_path:
        raise AttributeError("if is_plot is true then save_path must given")

    brain_count = 0
    csf_count = 0
    ok_slice = 0
    new_save_path = save_path / "main"
    new_img_path = save_path / "img"
    new_mask_path = save_path / "mask"

    for i in range(gt.shape[2]):
        print(i, end=" ", flush=True)
        contoured_bone = processing_bone(bone[..., i])
        brain_from_contoured_bone = np.sum(contoured_bone)
        if brain_from_contoured_bone > PIXEL_THRESHOLD:
            ok_slice += 1
            brain_count += brain_from_contoured_bone
            csf_count += np.sum(csf[..., i])
            if is_plot:
                new_file_name = ("%s_%d.png" % (file_name, i))
                new_save_file = new_save_path / new_file_name
                new_img_file = new_img_path / new_file_name
                new_mask_file = new_mask_path / new_file_name
                mask = np.where(contoured_bone == 1, 2, 0)
                mask = np.where(csf[..., i] == 1, 1, mask)
                plot_compair_each_slice(
                    gt[..., i], mask, ax3_title["v4"], str(new_save_file))
                cv2.imwrite(str(new_mask_file), mask)
                cv2.imwrite(str(new_img_file), gt[..., i])

    if brain_count != 0:
        csf_brain_ratio = csf_count / brain_count
    else:
        csf_brain_ratio = 0
    return csf_brain_ratio, csf_count, brain_count, ok_slice, gt.shape[2]


def cal_csf_to_brain_ratio_v4_2(gt, gray, white, csf, bone, *args, is_plot=False, save_path=None, file_name=None, **kwargs):
    # expected bone that has been windowing from gt >= 300
    if is_plot and not save_path:
        raise AttributeError("if is_plot is true then save_path must given")

    gt_nifti = kwargs["gt_nifti"]
    brain_count = 0
    csf_count = 0
    ok_slice = 0
#     print('test: ',csf.shape)
    new_mask = np.zeros_like(gt, dtype="uint8")
    for i in range(gt.shape[2]):
        print(i, end=" ", flush=True)
        contoured_bone = processing_bone(bone[..., i])
        brain_from_contoured_bone = np.sum(contoured_bone)
#         print('brain_from_contoured_bone : ',brain_from_contoured_bone)
#         if brain_from_contoured_bone > PIXEL_THRESHOLD:
        if brain_from_contoured_bone:
            ok_slice += 1
            brain_count += brain_from_contoured_bone
            csf_count += np.sum(csf[..., i])
            new_mask[..., i] = np.where(
                contoured_bone > 0, 2, new_mask[..., i])  # order matter
            new_mask[..., i] = np.where(csf[..., i] > 0, 1, new_mask[..., i])

    if is_plot:
        new_file_name = ("%s.nii" % (file_name))
        new_mask_file = save_path / new_file_name
        nib.save(nib.Nifti1Image(new_mask, affine=gt_nifti.affine,
                 header=gt_nifti.header), new_mask_file)

    if brain_count != 0:
        csf_brain_ratio = csf_count / brain_count
    else:
        csf_brain_ratio = 0
    return csf_brain_ratio, csf_count, brain_count, ok_slice, gt.shape[2]


def cal_csf_to_brain_ratio_v4_3code(gt, gray, white, csf, bone, *args, is_plot=False, save_path=None, file_name=None, **kwargs):
    # sum brain from c1 c2 c3
    if is_plot and not save_path:
        raise AttributeError("if is_plot is true then save_path must given")

    gt_nifti = kwargs["gt_nifti"]
    brain_count = 0
    csf_count = 0
    ok_slice = 0
    new_mask = np.zeros_like(gt, dtype="uint8")
    csf_mask = np.where(csf >= 1, 1, 0)
    white_mask = np.where(white >= 1.0, 1, 0)
    gray_mask = np.where(gray >= 1.0, 1, 0)
    for i in range(gt.shape[2]):
        print(i, end=" ", flush=True)
        contoured_bone = processing_bone(bone[..., i])
        ok_slice += 1
        brain_count += np.sum(gray_mask[..., i]) + \
            np.sum(white_mask[..., i])+np.sum(csf_mask[..., i])
        csf_count += np.sum(csf_mask[..., i])
        new_mask[contoured_bone > 0, i] = 2
        new_mask[gray_mask[..., i] > 0, i] = 2
        new_mask[white_mask[..., i] > 0, i] = 2
        new_mask[csf_mask[..., i] > 0, i] = 1

    if is_plot:
        print(is_plot)
        new_file_name = ("%s.nii" % (file_name))
        new_mask_file = save_path / new_file_name
        nib.save(nib.Nifti1Image(new_mask, affine=gt_nifti.affine,
                 header=gt_nifti.header), new_mask_file)

    if brain_count != 0:
        csf_brain_ratio = csf_count / brain_count
    else:
        csf_brain_ratio = 0
    return csf_brain_ratio, csf_count, brain_count, ok_slice, gt.shape[2]


def cal_csf_to_brain_ratio_v4_3(gt, gray, white, csf, bone, *args, is_plot=False, save_path=None, file_name=None, **kwargs):
    # sum brain from c1 c2 c3
    if is_plot and not save_path:
        raise AttributeError("if is_plot is true then save_path must given")

    gt_nifti = kwargs["gt_nifti"]
    brain_count = 0
    csf_count = 0
    ok_slice = 0
    new_mask = np.zeros_like(gt, dtype="uint8")
    csf_mask = np.where(csf >= 1, 1, 0)
    white_mask = np.where(white >= 1.0, 1, 0)
    gray_mask = np.where(gray >= 1.0, 1, 0)
    for i in range(gt.shape[2]):
        print(i, end=" ", flush=True)
        ok_slice += 1
        brain_count += np.sum(gray_mask[..., i]) + \
            np.sum(white_mask[..., i])+np.sum(csf_mask[..., i])
        csf_count += np.sum(csf_mask[..., i])
        new_mask[gray_mask[..., i] > 0, i] = 2
        new_mask[white_mask[..., i] > 0, i] = 2
        new_mask[csf_mask[..., i] > 0, i] = 1

    if is_plot:
        new_file_name = ("%s.nii" % (file_name))
        new_mask_file = save_path / new_file_name
        nib.save(nib.Nifti1Image(new_mask, affine=gt_nifti.affine,
                 header=gt_nifti.header), new_mask_file)

    if brain_count != 0:
        csf_brain_ratio = csf_count / brain_count
    else:
        csf_brain_ratio = 0
    return csf_brain_ratio, csf_count, brain_count, ok_slice, gt.shape[2]


# from model
def cal_csf_to_brain_ratio_from_model_output_v4_2(gt, csf_and_brain, bone_from_spm):
    # expected bone that has been windowing from gt >= 300
    brain_count = 0
    csf_count = 0
    ok_slice = 0

    ok_slice += gt.shape[2]
    width, height, channels = gt.shape
    for i in range(channels):
        print(i, end=" ", flush=True)
        bone_rescaled = cv2.resize(np.ones(
            (width, height, 3), dtype="uint8") * bone_from_spm[..., i, np.newaxis], (224, 224))[..., 0]
        csf_and_brain[..., i] = np.where(
            bone_rescaled > 0, 0, csf_and_brain[..., i])
        csf_count += np.sum(np.where(csf_and_brain[..., i] == 1, 1, 0))
        brain_count += np.sum(np.where(csf_and_brain[..., i] == 2, 1, 0))
    print()

    if brain_count != 0:
        csf_brain_ratio = csf_count / brain_count
    else:
        csf_brain_ratio = 0
    return csf_brain_ratio, csf_count, brain_count, ok_slice, gt.shape[2]


def cal_csf_to_brain_ratio_v5(gt, gray, white, csf, bone, brain, is_plot=False, save_path=None, file_name=None):
    # brain is hard-coded gt windowed [0, 90]
    if is_plot and not save_path and not file_name:
        raise AttributeError(
            "if is_plot is true then save_path and file_name must given")

    brain_count = 0
    csf_count = 0
    ok_slice = 0
    new_save_path = save_path / "main"
    new_csf_path = save_path / "csf"
    new_bone_path = save_path / "bone"
    for i in range(gt.shape[2]):
        print(i, end=" ", flush=True)
        brain_count_i = np.sum(brain[..., i])

        if brain_count_i > PIXEL_THRESHOLD:
            ok_slice += 1
            brain_count += brain_count_i
            csf_count += np.sum(csf[..., i])
            if is_plot:
                file_name = ("%s_%d.png" % (file_name, i))
                new_save_path = new_save_path / file_name
                new_csf_path = new_csf_path / file_name
                new_bone_path = new_bone_path / file_name
                mask = np.where(brain[..., i] == 1, 2, 0)
                mask = np.where(csf[..., i] == 1, 1, mask)
                plot_compair_each_slice(
                    gt[..., i], mask, ax3_title["v5"], str(new_save_path))
                cv2.imwrite(str(new_csf_path), csf[..., i])
                cv2.imwrite(str(new_csf_path), brain[..., i])

    if brain_count != 0:
        csf_brain_ratio = csf_count / brain_count
    else:
        csf_brain_ratio = 0
    return csf_brain_ratio, csf_count, brain_count, ok_slice, gt.shape[2]


def cal_csf_to_brain_ratio_v6(gt, gray, white, csf, bone, *args, is_plot=False, save_path=None, file_name=None, **kwargs):
    # expected bone that has been windowing from gt >= 300
    if is_plot and not save_path:
        raise AttributeError("if is_plot is true then save_path must given")

    gt_nifti = kwargs["gt_nifti"]
    brain_count = 0
    csf_count = 0
    ok_slice = 0

    spm = gray | white | csf
    new_mask = np.zeros_like(gt, dtype="uint8")
    for i in range(gt.shape[2]):
        print(i, end=" ", flush=True)
        contoured_bone = processing_bone(bone[..., i])
        brain_from_contoured_bone = np.sum(contoured_bone)
        spm_i = spm[..., i]
        brain_from_spm = np.sum(spm_i)

        csf_count += np.sum(csf[..., i])
        if brain_from_contoured_bone > brain_from_spm:
            ok_slice += 1
            brain_count += brain_from_contoured_bone
            new_mask[..., i] = np.where(
                contoured_bone > 0, 2, new_mask[..., i])  # order matter
            new_mask[..., i] = np.where(csf[..., i] > 0, 1, new_mask[..., i])

        else:
            ok_slice += 1
            brain_count += brain_from_spm
            new_mask[..., i] = np.where(
                spm_i > 0, 2, new_mask[..., i])  # order matter
            new_mask[..., i] = np.where(csf[..., i] > 0, 1, new_mask[..., i])
        if is_plot:
            new_img_name = ("%s_%d.png" % (file_name, i))
            plot_compair_each_slice(gt[..., i], new_mask[..., i], "spm and contoured bone", str(
                save_path/"main"/new_img_name))

    print()

    if is_plot and brain_count > PIXEL_THRESHOLD * 10:

        new_file_name = ("%s.nii" % (file_name))
        new_mask_file = save_path / "mask" / new_file_name
        nib.save(nib.Nifti1Image(new_mask, affine=gt_nifti.affine,
                 header=gt_nifti.header), new_mask_file)

    if brain_count != 0:
        csf_brain_ratio = csf_count / brain_count
    else:
        csf_brain_ratio = 0
    return csf_brain_ratio, csf_count, brain_count, ok_slice, gt.shape[2]


COLOR = [1, 1, 1]


def processing_bone(bone_slice):
    # fully prevent side-effect
    new_bone_slice = bone_slice.copy().astype("uint8") * 255
    contours, hierarchy = cv2.findContours(
        new_bone_slice, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    drawing = np.zeros(shape=new_bone_slice.shape, dtype="uint8")
    cv2.drawContours(drawing, contours, -1, COLOR,
                     cv2.FILLED, cv2.LINE_8, hierarchy, 0)

    drawing = drawing.astype("int8") - bone_slice.astype("int8")
    drawing = np.where(drawing < 0, 0, drawing)
    return drawing


COLUMNS = ["gray_path", "white_path", "csf_path", "bone_path", "file_name",
           "csf_pixel_count", "brain_pixel_count", "csf_to_brain_ratio", "ok_slice", "all_slice"]


def list_all_ratio(gt_dir, gray_dir, white_dir, csf_dir, bone_dir, is_plot=False, save_dir=".", output_csv="test.csv"):
    """
    from nii mask
    """
    if is_plot and not save_dir:
        raise AttributeError("if is_plot is true then save_path must given")

    gt_path = Path(gt_dir).absolute()
    gray_path = Path(gray_dir).absolute()
    white_path = Path(white_dir).absolute()
    csf_path = Path(csf_dir).absolute()
    bone_path = Path(bone_dir).absolute()
    save_path = Path(save_dir).absolute()
    save_path.mkdir(exist_ok=True)
    df = pd.DataFrame(columns=COLUMNS)

    file_names = os.listdir(path=str(gt_path))

    for each_file in file_names:
        try:
            file_name = each_file.split(".nii")[0]
            gt_file = gt_path / each_file


            '''
                my folder has different names of files so I have 2 way to replace it
                this is 1st way
            '''
            gray_file = gray_path / Path('c1'+str(each_file)) 
            white_file = white_path / Path('c2'+str(each_file)) 
            csf_file = csf_path / Path('c3'+str(each_file))
            bone_file = bone_path / Path('c4'+str(each_file)) 

            '''
                the comment below is 2nd way just folder NPH/1NPH23
            '''
#                 gray_file = gray_path / Path(str(each_file))#.replace('s','c1')
#                 white_file = white_path / Path(str(each_file))#.replace('s','c2')
#                 csf_file = csf_path / Path(str(each_file))#.replace('s','c3')
#                 bone_file = bone_path / Path(str(each_file))#.replace('s','c4')
#                 print(gray_file,white_file,csf_file,bone_file)
            # save_folder = save_path / file_name # comment this line to get training set, or uncomment this line to get patient wise data
            save_folder = save_path
            print("reading file %s" % each_file, end=" ")

            gt_array, gt_nif = aux_func.load_nifti(str(gt_file), "float32")
            gray_array, _ = aux_func.load_nifti(str(gray_file), "float32")
            white_array, _ = aux_func.load_nifti(
                str(white_file), "float32")
            csf_array, _ = aux_func.load_nifti(str(csf_file), "float32")
            bone_array, _ = aux_func.load_nifti(str(bone_file), "float32")


        except Exception as e:
            print(e)
            continue

        # # for calculate ratio from spm (without using bone contour technique)
#         csf_to_brain_ratio, csf_count, brain_count, ok_slice, all_slice = cal_csf_to_brain_ratio_v1(
#             gt_array, gray_array, white_array, csf_array, bone_array, is_plot=is_plot, save_path=save_folder, file_name=file_name, gt_nifti=gt_nif)

        # # for calculate ratio and create .nii file (using bone contour technique for brain) for create label mask
        csf_to_brain_ratio, csf_count, brain_count, ok_slice, all_slice = cal_csf_to_brain_ratio_v4_3(
            gt_array, gray_array, white_array, csf_array, bone_array, is_plot=True, save_path=save_folder, file_name=file_name, gt_nifti=gt_nif)

        # for calculate ratio from model segmentation output (.nii)
#         csf_to_brain_ratio, csf_count, brain_count, ok_slice, all_slice = cal_csf_to_brain_ratio_from_model_output_v4_2(
#             gt_array, csf_and_brain=csf_array, bone_from_spm=bone_array)

        a_row = {
            "gray_path": str(gray_file),
            "white_path": str(white_file),
            "csf_path": str(csf_file),
            "bone_path": str(bone_file),
            "file_name": each_file,
            "csf_to_brain_ratio": csf_to_brain_ratio,
            "csf_pixel_count": csf_count,
            "brain_pixel_count": brain_count,
            "ok_slice": ok_slice,
            "all_slice": all_slice
        }
        df = df.append(a_row, ignore_index=True, sort=False)
        print()
    print(df)

    print("writing all ratio to file:", output_csv)
    df.to_csv(output_csv, encoding="utf-8")


def find_contours_in_bone(bone_dir):

    bone_array, _ = aux_func.load_nifti(str(bone_dir), "uint8")
    bone_array = bone_array * 255
    Path("./data_edge").mkdir(exist_ok=True)

    for s in range(bone_array.shape[2]):
        bone_slice = bone_array[:, :, s]
        new_bone_slice = bone_slice.copy()

        contours, hierarchy = cv2.findContours(
            new_bone_slice, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)
        drawing1 = np.zeros(shape=new_bone_slice.shape, dtype="uint8")
        drawing2 = np.zeros(shape=(*new_bone_slice.shape, 3), dtype="uint8")
        cv2.drawContours(drawing1, contours, -1, COLOR,
                         cv2.FILLED, cv2.LINE_8, hierarchy, 0)
        cv2.drawContours(drawing2, contours, -1, COLOR,
                         8, cv2.LINE_8, hierarchy, 0)

        fig = plt.figure(figsize=(10, 5))
        ax1 = fig.add_subplot(1, 3, 1)
        ax1.axis("off")
        ax1.set_title("bone mask")
        ax1.imshow(bone_slice, cmap="gray")

        new_bone_slice = np.ones(
            shape=(*bone_slice.shape, 3), dtype="uint8") * bone_slice[..., np.newaxis]
        new_bone_slice[..., 0] = np.where(
            drawing1 > 0, drawing1, new_bone_slice[..., 0])
        ax2 = fig.add_subplot(1, 3, 2)
        ax2.axis("off")
        ax2.set_title("contours (filled) with bone mask")
        ax2.imshow(new_bone_slice, cmap="gray")

        ax3 = fig.add_subplot(1, 3, 3)
        ax3.axis("off")
        ax3.set_title("contours")
        ax3.imshow(drawing2, cmap="gray")
        fig.savefig("data_edge/%d.png" % s)
        plt.close(fig)


def main():
    '''
    can define folder path by structure as your directory structure
    my structure is
    root
    |---normal
    |   |---110
    |   |   |---1 (gray matter)
    |   |   |---2 (white matter)
    |   |   |---3 (csf)
    |   |   |---4 (bone)
    |   |   |---s (raw data)
    |---nph (same as above)
    '''
    
    folder_name = "3nii_norm68"
#     folder_name = "4norm14112022_59"
#     folder_name = "norm125"
    type_folder = "nph"
#     type_folder = "normal"
    gt_path = Path(
        f"/data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/{type_folder}/{folder_name}/s")
    gray_path = Path(
        f"/data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/{type_folder}/{folder_name}/1")
    white_path = Path(
        f"/data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/{type_folder}/{folder_name}/2")
    csf_path = Path(
        f"/data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/{type_folder}/{folder_name}/3")
    bone_path = Path(
        f"/data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/{type_folder}/{folder_name}/4")
    save_path = f"/data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/{type_folder}/{folder_name}/test_segment_mask_confidence1"

    is_plot = True

    output_file = f"./csv/segment_mask_{folder_name}_test_confidence1.csv"
    list_all_ratio(gt_path, gray_path, white_path, csf_path,
                   bone_path, is_plot, save_path, output_file)


if __name__ == "__main__":
    main()
